# Notebook 기본 세팅

In [1]:
# Constant 선언

# 프로젝트 루트 디렉토리를 식별하기 위한 마커 파일 이름
ROOT_MARKER = "pyproject.toml"

# 한글 표시를 위한 나눔바른고딕 폰트 파일 이름
# matplotlib 의 font_manager 에 실제 폰트 파일의 위치를 넣어주어야 한다.
KOREAN_FONT_FILE = "NanumBarunGothic.ttf"

# matplotlib 에서는 font-family 의 이름으로 font 를 설정한다.
# 그래서 font 파일 그 자체가 아니라, 그 파일의 family 이름을 적어준다.
KOREAN_FONT_FAMILY = "NanumBarunGothic"

# 참고
# Font Family 와 Font File 의 차이는,
# Font Family 는 비슷한 디자인 특성을 공유하는 글꼴 그룹을 의미한다.
#
# 예를 들어 '나눔바른고딕' 폰트 패밀리는 일반(Regular), 굵게(Bold), 기울임(Italic) 등 여러 스타일을 포함할 수 있다.
# 반면, 폰트 파일(.ttf, .otf 등)은 이러한 폰트의 하나의 스타일이 저장된 실제 파일이다.
#
# 이 프로젝트에서는 폰트 용량을 줄이기 위해 일반(Regular) 인 NanumBarunGothic.ttf 만 사용한다.

In [2]:
# 프로젝트 root 를 sys.path 에 추가해서 import 구문을 사용하기 쉽게
from pathlib import Path


def find_project_root() -> Path:
    """
    pyproject.toml 파일을 기준으로 루트 디렉토리를 찾는다.
    :return: Path: 프로젝트 루트 디렉토리 경로
    """

    current_path = Path().resolve()

    while current_path != current_path.parent:
        if (current_path / ROOT_MARKER).exists():
            return current_path

        current_path = current_path.parent

    raise FileNotFoundError("프로젝트 루트 디렉토리를 찾을 수 없습니다.")


ROOT_DIR = find_project_root()

In [3]:
# matplotlib 의 한글 font 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt


FONTS_DATA_DIR = ROOT_DIR / "notebooks" / "fonts"


def setup_korean_font():
    font_path = FONTS_DATA_DIR / KOREAN_FONT_FILE
    fm.fontManager.addfont(font_path)

    # 폰트 설정
    plt.rcParams["font.family"] = KOREAN_FONT_FAMILY
    plt.rcParams["axes.unicode_minus"] = False


setup_korean_font()

# 데이터 수집
1.	weather_station_list.csv에 정의된 전체 지점(stnId)을 기준으로 기상청 API에서 날씨 데이터를 수집
2.	JSON 응답 데이터를 DataFrame으로 변환하고, 기존 데이터(weather_raw.csv)가 있다면 병합 
3.	최신 버전으로 weather_raw.csv 덮어쓰기 저장

In [4]:
import pandas as pd
import requests
import os
from io import StringIO
from datetime import datetime
from time import sleep
import boto3
from botocore.client import Config
import math

# 환경변수 불러오기
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
NCLOUD_ACCESS_KEY = os.getenv("NCLOUD_ACCESS_KEY")
NCLOUD_SECRET_KEY = os.getenv("NCLOUD_SECRET_KEY")
NCLOUD_STORAGE_REGION = os.getenv("NCLOUD_STORAGE_REGION")
NCLOUD_STORAGE_BUCKET = os.getenv("NCLOUD_STORAGE_BUCKET")
NCLOUD_STORAGE_ENDPOINT_URL = os.getenv("NCLOUD_STORAGE_ENDPOINT_URL")

# S3 클라이언트 생성
s3_client = boto3.client(
    "s3",
    endpoint_url=NCLOUD_STORAGE_ENDPOINT_URL,
    aws_access_key_id=NCLOUD_ACCESS_KEY,
    aws_secret_access_key=NCLOUD_SECRET_KEY,
    region_name=NCLOUD_STORAGE_REGION,
    config=Config(signature_version="s3v4"),
)

# 데이터 저장 경로
DATASETS_DIR = "datasets"

In [5]:
# S3에서 CSV 읽는 함수
def read_csv_as_dataframe(s3_key: str) -> pd.DataFrame:
    response = s3_client.get_object(Bucket=NCLOUD_STORAGE_BUCKET, Key=s3_key)
    return pd.read_csv(response["Body"])

In [6]:
def fetch_weather_data_to_s3_with_pagination(
    start_date: str,
    end_date: str,
    station_csv_path: str,
    s3_filename: str,
):
    """
    전체 지점의 날씨 데이터를 페이지네이션 방식으로 수집하고 S3에 저장 (기존 데이터 병합 없이 덮어쓰기)
    """
    base_url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
    weather_api_key = os.getenv("WEATHER_API_KEY")

    # 1. 전체 지점 목록 불러오기
    station_df = pd.read_csv(station_csv_path)
    station_ids = station_df["stnId"].dropna().astype(str).unique()

    all_data = []

    for stn_id in station_ids:
        print(f"\n📍 지점 {stn_id} 데이터 수집 시작")
        common_params = {
            "serviceKey": weather_api_key,
            "pageNo": "1",
            "numOfRows": "999",
            "dataType": "JSON",
            "dataCd": "ASOS",
            "dateCd": "DAY",
            "startDt": start_date,
            "endDt": end_date,
            "stnIds": stn_id,
        }

        # 최대 pageNo 계산
        try:
            response = requests.get(base_url, params=common_params, timeout=20)
            response.raise_for_status()
            json_data = response.json()

            if 'response' in json_data and 'body' in json_data['response']:
                body = json_data['response']['body']
                total_count = int(body['totalCount'])
                max_page_no = math.ceil(total_count / int(common_params["numOfRows"]))
                print(f"📄 총 페이지 수: {max_page_no}")
            else:
                raise ValueError("❌ 응답에 'body' 없음 또는 데이터 없음")

        except Exception as e:
            print(f"❌ 페이지 수 계산 실패 - 지점 {stn_id}: {e}")
            continue

        # 페이지별 데이터 수집
        for page in range(1, max_page_no + 1):
            common_params["pageNo"] = str(page)
            try:
                response = requests.get(base_url, params=common_params, timeout=20)
                response.raise_for_status()
                items = response.json()['response']['body']['items']['item']
                df = pd.DataFrame(items)
                df["stnId"] = stn_id
                all_data.append(df)
                print(f"✅ {stn_id} - 페이지 {page} 수집 완료")
                sleep(1)
            except Exception as e:
                print(f"⚠️ {stn_id} - 페이지 {page} 수집 실패: {e}")
                continue

    if not all_data:
        print("❌ 데이터 수집 결과가 비어 있습니다. 종료합니다.")
        return

    # 데이터 병합
    full_df = pd.concat(all_data, ignore_index=True)
    print(f"\n✅ 전체 데이터 수집 완료. 총 샘플 수: {full_df.shape[0]}")

    # 로컬 CSV 저장
    full_df.to_csv("weather_fulldata.csv", index=False)
    print("✅ 수집한 전체 데이터를 weather_fulldata.csv 파일로 저장했습니다.")

    # # S3 업로드
    # csv_buffer = StringIO()
    # full_df.to_csv(csv_buffer, index=False)
    # s3_key = f"{DATASETS_DIR}/{s3_filename}"
    # s3_client.put_object(Bucket=NCLOUD_STORAGE_BUCKET, Key=s3_key, Body=csv_buffer.getvalue())
    # print(f"✅ S3 저장 완료: {s3_key}")

In [7]:
from datetime import datetime, timedelta

# 어제 날짜를 YYYYMMDD 형식 문자열로 생성
yesterday_str = (datetime.today() - timedelta(days=1)).strftime("%Y%m%d")

# 함수 호출 예시
fetch_weather_data_to_s3_with_pagination(
    start_date="20200101",
    end_date=yesterday_str,
    station_csv_path="./weather_station_list.csv",
    s3_filename="weather_all_station.csv"
)


📍 지점 90 데이터 수집 시작
📄 총 페이지 수: 2
✅ 90 - 페이지 1 수집 완료
✅ 90 - 페이지 2 수집 완료

📍 지점 93 데이터 수집 시작
📄 총 페이지 수: 2
✅ 93 - 페이지 1 수집 완료
✅ 93 - 페이지 2 수집 완료

📍 지점 95 데이터 수집 시작
📄 총 페이지 수: 2
✅ 95 - 페이지 1 수집 완료
✅ 95 - 페이지 2 수집 완료

📍 지점 98 데이터 수집 시작
📄 총 페이지 수: 2
✅ 98 - 페이지 1 수집 완료
✅ 98 - 페이지 2 수집 완료

📍 지점 99 데이터 수집 시작
📄 총 페이지 수: 2
✅ 99 - 페이지 1 수집 완료
✅ 99 - 페이지 2 수집 완료

📍 지점 100 데이터 수집 시작
📄 총 페이지 수: 2
✅ 100 - 페이지 1 수집 완료
✅ 100 - 페이지 2 수집 완료

📍 지점 101 데이터 수집 시작
📄 총 페이지 수: 2
✅ 101 - 페이지 1 수집 완료
✅ 101 - 페이지 2 수집 완료

📍 지점 102 데이터 수집 시작
📄 총 페이지 수: 2
✅ 102 - 페이지 1 수집 완료
✅ 102 - 페이지 2 수집 완료

📍 지점 104 데이터 수집 시작
📄 총 페이지 수: 2
✅ 104 - 페이지 1 수집 완료
✅ 104 - 페이지 2 수집 완료

📍 지점 105 데이터 수집 시작
📄 총 페이지 수: 2
✅ 105 - 페이지 1 수집 완료
✅ 105 - 페이지 2 수집 완료

📍 지점 106 데이터 수집 시작
📄 총 페이지 수: 2
✅ 106 - 페이지 1 수집 완료
✅ 106 - 페이지 2 수집 완료

📍 지점 108 데이터 수집 시작
📄 총 페이지 수: 2
✅ 108 - 페이지 1 수집 완료
✅ 108 - 페이지 2 수집 완료

📍 지점 112 데이터 수집 시작
📄 총 페이지 수: 2
✅ 112 - 페이지 1 수집 완료
✅ 112 - 페이지 2 수집 완료

📍 지점 114 데이터 수집 시작
📄 총 페이지 수: 2
✅ 114 - 페이지 1 수집 완료
✅ 114 - 페이지 2

In [ ]:
# 업로드된 파일 삭제
s3_client.delete_object(Bucket=NCLOUD_STORAGE_BUCKET, Key=f"{DATASETS_DIR}/weather_all_station.csv")